<a href="https://colab.research.google.com/github/zzyer/NewGrad-2021/blob/master/Task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import json

In [ ]:
# Load the config
config_filePath = '/content/drive/MyDrive/Colab Notebooks/XTX_Market/Task2/mm/mm_config.json'
with open(config_filePath) as f:
  config = json.load(f)
f.close()

# Load the market making data
mm_data_filePath = '/content/drive/MyDrive/Colab Notebooks/XTX_Market/Task2/mm/mm.csv' 
mm_data = pd.read_csv(mm_data_filePath)

In [ ]:
# Get the time requirement of each trading session in seconds
for k, v in config.items():
  config[k]['req_time_sec'] = (pd.to_datetime(v['end']) - pd.to_datetime(v['start'])).total_seconds() * v['requirement_time']

In [ ]:
# Unroll the data by time interval into data by seconds
mm_data['time_seconds'] = mm_data.apply(lambda df: pd.date_range(df['entered_ts'], df['completed_ts'], freq='S', closed='left'), axis=1)
mm_data_unroll = mm_data.explode('time_seconds').reset_index(drop=True)

Buy_mm_data = mm_data_unroll[mm_data_unroll['side'] == 'B'].groupby(['instrument', 'time_seconds']) \
                                                           .agg({'price': 'min', 'size': 'sum'}) \
                                                           .rename(columns={"price": "bid_price_min", "size": "bid_size_sum"})

Sell_mm_data = mm_data_unroll[mm_data_unroll['side'] == 'S'].groupby(['instrument', 'time_seconds']) \
                                                            .agg({'price': 'max', 'size': 'sum'}) \
                                                            .rename(columns={"price": "ask_price_max", "size": "ask_size_sum"})

mm_data_two_side = pd.merge(Buy_mm_data, Sell_mm_data, how="left", left_index=True, right_index=True)
mm_data_two_side

bid_price_min  bid_size_sum  ask_price_max  \
instrument time_seconds                                                      
A          2020-02-15 05:00:00          386.0           405          395.0   
           2020-02-15 05:00:01          386.0           405          395.0   
           2020-02-15 05:00:02          386.0           405          395.0   
           2020-02-15 05:00:03          386.0           405          395.0   
           2020-02-15 05:00:04          386.0           405          395.0   
...                                       ...           ...            ...   
C          2020-02-15 12:59:55          582.0           406          588.0   
           2020-02-15 12:59:56          582.0           406          588.0   
           2020-02-15 12:59:57          582.0           406          588.0   
           2020-02-15 12:59:58          582.0           406          588.0   
           2020-02-15 12:59:59          582.0           406          588.0   

                                ask_size_sum  
instrument time_seconds                       
A          2020-02-15 05:00:00           407  
           2020-02-15 05:00:01           407  
           2020-02-15 05:00:02           407  
           2020-02-15 05:00:03           407  
           2020-02-15 05:00:04           407  
...                                      ...  
C          2020-02-15 12:59:55           400  
           2020-02-15 12:59:56           400  
           2020-02-15 12:59:57           400  
           2020-02-15 12:59:58           400  
           2020-02-15 12:59:59           400  

[61200 rows x 4 columns]

In [ ]:
# Calculate total time for which the maximum spread with minimum size was shown for, in seconds.
mm_data_two_side['max_spread'] = mm_data_two_side['ask_price_max'] - mm_data_two_side['bid_price_min']
mm_data_two_side['if_meet_requirement'] = mm_data_two_side.apply(lambda row:
                                                                 row['max_spread'] <= config[row.name[0]]['requirement_spread'] and
                                                                 row['bid_size_sum'] >= config[row.name[0]]['requirement_size'] and
                                                                 row['ask_size_sum'] >= config[row.name[0]]['requirement_size'], axis=1)
res = mm_data_two_side.groupby('instrument') \
                      .agg({'if_meet_requirement': 'sum'}) \
                      .rename(columns={"if_meet_requirement": 'qualified_seconds'})
res['required_seconds'] = [config[i]['req_time_sec'] for i in res.index]
res['if_meet_timeReq'] = res['qualified_seconds'] >= res['required_seconds']
res

,qualified_seconds,required_seconds,if_meet_timeReq
instrument,,,
A,5820,9000.0,False
B,6360,14400.0,False
C,8760,7560.0,True
